In [1]:
import numpy as np
import re
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import pandas as pd
# fix random seed for reproducibility
np.random.seed(7)

Using Theano backend.


### Preparing Training data

In [2]:
# Load training dataset
X_train, y_train = [], []
f = open("./PreProcess/preprocessed_filtered.txt")
for l in f:
    if l.count(" ") == 0 :
        continue;
    key, val = l.split()
    key = re.sub('[^0-9a-zA-Z]+', '', key)
    if len(key.strip())==0:
        continue;
    X_train += [ key.lower().strip() ]
    y_train += [ val ]

In [3]:
d = {}
f = open("./Word_Embeedings/Text_filtered.vectors.txt")
for l in f:
    words = l.split()
    d[ str(words[0]).strip() ] = [ float(v) for v in words[1:] ]

In [4]:
found = 0
notFound = []
newX = []
newY = []
for i in xrange( len(X_train) ):
    key = str(X_train[i]).strip()
    key = key.lower();
    if key in d:
        newX += [ d[key] ]
        newY += [ 0 if y_train[i].strip() == 'none' else 1]
        found += 1
    else:
        notFound += [key]
print "Number of words for training: ", found, " out of ", len(X_train)

Number of words for training:  48335  out of  52120


In [6]:
newX = np.array( [ np.array(x) for x in newX ] )
newY = np.array( [ np.array(x) for x in newY ] )
print newX.shape, newY.shape

(48335, 10) (48335,)


In [7]:
def create_dataset(datasetX, datasetY, window=5):
    dataX, dataY = [], []
    for i in range(len(datasetX)-window):
        dataX.append(datasetX[i:(i+window)])
        dataY.append(datasetY[i + window/2])
    return np.array(dataX), np.array(dataY)

In [8]:
trainX, trainY = create_dataset(newX, newY, 5 )

In [104]:
trainX.shape

(48330, 5, 10)

# Preparing Test data

In [18]:
# Load test dataset
x_test, y_test = [], []
f = open("./PreProcess/test_preprocessed.txt")
for l in f:
    if l.count(" ") == 0 :
        continue;
    l = l.split()
    key, val = l[0], l[1]
    key = re.sub('[^0-9a-zA-Z]+', '', key)
    if len(key.strip())==0:
        continue;
    x_test += [ key.lower().strip() ]
    y_test += [ val ]

In [19]:
found = 0
notFound = []
testX = []
testY = []
for i in xrange( len(x_test) ):
    key = str(x_test[i]).strip()
    key = key.lower();
    if key in d:
        testX += [ d[key] ]
        testY += [ 0 if y_test[i].strip() == 'none' else 1]
        found += 1
    else:
        notFound += [key]
print "Number of words for training: ", found, " out of ", len(testX)

Number of words for training:  2872  out of  2872


In [20]:
newX = np.array( [ np.array(x) for x in testX ] )
newY = np.array( [ np.array(x) for x in testY ] )
print newX.shape, newY.shape

(2872, 10) (2872,)


## Building model

In [121]:
model = Sequential()
model.add( LSTM(32, input_dim=10) )
model.add(Dropout(0.3))
model.add( Dense(1) )
model.compile( loss="mean_squared_error", optimizer="adam", metrics=['accuracy'] )
model.fit( trainX, trainY, nb_epoch=100, batch_size=10 )

Epoch 1/100
48330/48330 [==============================] - 26s - loss: 0.2114 - acc: 0.6684    
Epoch 2/100
48330/48330 [==============================] - 34s - loss: 0.1980 - acc: 0.6892    
Epoch 3/100
48330/48330 [==============================] - 33s - loss: 0.1947 - acc: 0.6945    
Epoch 4/100
48330/48330 [==============================] - 34s - loss: 0.1927 - acc: 0.6996    
Epoch 5/100
48330/48330 [==============================] - 33s - loss: 0.1914 - acc: 0.7030    
Epoch 6/100
48330/48330 [==============================] - 34s - loss: 0.1904 - acc: 0.7033    
Epoch 7/100
48330/48330 [==============================] - 34s - loss: 0.1897 - acc: 0.7056    
Epoch 8/100
48330/48330 [==============================] - 34s - loss: 0.1881 - acc: 0.7090    
Epoch 9/100
48330/48330 [==============================] - 35s - loss: 0.1876 - acc: 0.7107    
Epoch 10/100
48330/48330 [==============================] - 35s - loss: 0.1871 - acc: 0.7118    
Epoch 11/100
48330/48330 [=============

In [ ]:
stateful_model = Sequential()
stateful_model.add( LSTM(10, batch_input_shape=(1, 48330, 5, 10), stateful=True, return_sequences=True ) )
stateful_model.add(Dropout(0.3))
stateful_model.add( LSTM(10, stateful=True ) )
stateful_model.add(Dropout(0.3))
stateful_model.add( Dense(1) ) #0.1980
stateful_model.compile( loss="mean_squared_error", optimizer="adam", metrics=['accuracy'] )
for i in xrange(10):
    print "epoch: ", i
    stateful_model.fit( state_trainX, trainY, nb_epoch=1, batch_size=5, shuffle=False )
    stateful_model.reset_states()

In [110]:
trainScore = model.evaluate(trainX, trainY, batch_size=1, verbose=0 )
print trainScore

[0.16058620648470551, 0.76269397889509616]


# Evaluate model on Test data

In [124]:
testScore = model.evaluate(testX, testY, batch_size=10, verbose=0 )
print testScore

[0.32856243809299845, 0.47331705615626091]


In [141]:
predictions = model.predict(testX)
predictions = [ 1 if i>=0.5 else 0 for i in predictions ]

In [142]:
predOnes = 0
for i in xrange(len(predictions)):
    predOnes += (1 if predictions[i]==1 else 0 )
    
groundOnes = 0
for i in xrange(len(testY)):
    groundOnes += (1 if testY[i]==1 else 0 )
    
correctPredicOnes = 0
for i in xrange(len(predictions)):
    correctPredicOnes += (1 if predictions[i]==testY[i] else 0 )
    
print predOnes, groundOnes, correctPredicOnes

1357 2867 1357


In [146]:
p = 1.0*correctPredicOnes / predOnes
r = 1.0*correctPredicOnes / groundOnes
print "Precision: ", p
print "Recall: ", r
print "F-Score: ", 2 * p* r/ (p+r)

Precision:  1.0
Recall:  0.473317056156
F-Score:  0.642518939394
